In [64]:
import os
import csv
import json

import librosa

In [ ]:
sound_emotion_path = "../../databases/TESS_and_RAVDESS/"
data = {
    "mapping": [],
    "mfcc": [],
    "label": []
}

In [61]:
def process_sound(filename, dirpath, emotion, sample_rate, n_mfcc, n_fft, hop_length):
    file_path = os.path.join(dirpath, filename)
    # sample rate is per-second, signal.size/sample rate = time
    signal, sr = librosa.load(file_path, sr=sample_rate)
    # Option to round this so we get more samples, but having consistent 1 second samples is important too
    intervals = int(signal.size / sample_rate)
    for x in range(intervals):
        start = x * sample_rate
        stop = start + sample_rate
        mfcc = librosa.feature.mfcc(signal[start:stop], sr=sample_rate, n_fft=n_fft, hop_length=hop_length, n_mfcc=n_mfcc)
        # Easier to work with
        # TODO normalize!!!!! we have ranges from -1100 to 
        mfcc = mfcc.T
        data["mfcc"].append(mfcc.tolist())
        data["label"].append(emotion)
        # The length of each mfcc should be equal because we need that in order to properly train on a 1 second-sample with a 
        # sample rate of 22050. If we change anything about the n_fft, or hop_length, we must re-process everything from scratch
        # print(len(mfcc))


In [62]:
def save_mfcc(dataset_path, n_mfcc=13, n_fft=1024, hop_length=256, sample_rate=22050):
    # count: 0=Angry, 1=Disgusted, 2=Fearful, 3=Happy, 4=Netural, 5=Sad, 6=Surprised
    for count, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
        # print(filenames)
        # print(dirnames)
        print(dirpath)
        if dirpath is not sound_emotion_path:
            if "Angry" in dirpath:
                for f in filenames:
                    process_sound(f, dirpath, "anger_disgust", sample_rate, n_mfcc, n_fft, hop_length)

save_mfcc(sound_emotion_path)

../../databases/TESS_and_RAVDESS/
../../databases/TESS_and_RAVDESS/Angry
../../databases/TESS_and_RAVDESS/Disgusted
../../databases/TESS_and_RAVDESS/Fearful
../../databases/TESS_and_RAVDESS/Happy
../../databases/TESS_and_RAVDESS/Neutral
../../databases/TESS_and_RAVDESS/Sad
../../databases/TESS_and_RAVDESS/Suprised


In [65]:
with open("./tess_ravdess.json", "w") as fp:
    json.dump(data, fp, indent=4)